In [1]:
from DataSynthesizer.DataDescriber import DataDescriber
from DataSynthesizer.DataGenerator import DataGenerator
from DataSynthesizer.ModelInspector import ModelInspector
from DataSynthesizer.lib.utils import read_json_file, display_bayesian_network

import pandas as pd
import numpy as np

In [2]:
# input dataset
input_data = 'mimic_iv_subset.csv'
# location of two output files
mode = 'random_mode'
description_file = './out/description.json'
synthetic_data = './out/sythetic_data.csv'

In [3]:
# An attribute is categorical if its domain size is less than this threshold.
# Here modify the threshold to adapt to the domain size of "education" (which is 14 in input dataset).
threshold_value = 20 

In [4]:
describer = DataDescriber(category_threshold=threshold_value)
describer.describe_dataset_in_random_mode(input_data)
describer.save_dataset_description_to_file(description_file)

In [26]:
PATIENT_VARS = [
    'admission_age', 'hospital_expire_flag', 'icu_expire_flag',
    'admission_type_Medical', 'admission_type_ScheduledSurgical',
    'admission_type_UnscheduledSurgical', 'charlson_comorbidity_index'
]

DUMMY_VARS = [[
            'admission_type_Medical', 'admission_type_ScheduledSurgical',
            'admission_type_UnscheduledSurgical']]


def generate_patient(description_file, los=None):
    if los is None:
        los = np.random.randint(50, high=200)
    generator = DataGenerator()
    generator.generate_dataset_in_random_mode(los, description_file)
    synthetic_dataset = generator.synthetic_dataset.copy()
    selected_ix = np.random.choice(range(synthetic_dataset.shape[0]))
    for v in PATIENT_VARS:
        synthetic_dataset.loc[v] = synthetic_dataset.iloc[selected_ix][v]
    for d_v in DUMMY_VARS:
        if synthetic_dataset[d_v].sum(axis=1).max() > 1:
            selected_var = np.random.choice(d_v)
            synthetic_dataset.loc[d_v] = 0
            synthetic_dataset.loc[selected_var] = 1
    synthetic_dataset['hr'] = range(synthetic_dataset.shape[0])
    synthetic_dataset['icu_expire_flag'] = synthetic_dataset['icu_expire_flag'].clip(0,1)
    synthetic_dataset['hospital_expire_flag'] = synthetic_dataset['hospital_expire_flag'].clip(0,1)
    if synthetic_dataset['icu_expire_flag'].max() == 1:
        synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
    return synthetic_dataset

In [27]:
patient = generate_patient(description_file, los=50)

/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1


In [28]:
patient.head()

,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,pao2fio2ratio_novent,pao2fio2ratio_vent,gcs_min,bilirubin_max,creatinine_max,platelet_min,admission_age,hospital_expire_flag,icu_expire_flag,admission_type_Medical,admission_type_ScheduledSurgical,admission_type_UnscheduledSurgical,charlson_comorbidity_index,hr
0,54.881350,57.019677,67.781654,14.944830,31.179588,35.561274,90.655550,11.0,86.302262,92.687909,2.0,97.0,1,1.0,1.0,1.0,0.0,12.0,0
1,71.518937,43.860151,27.000797,86.812606,69.634349,94.043195,77.404733,4.0,15.763228,85.394012,89.0,12.0,1,1.0,1.0,1.0,1.0,18.0,1
2,60.276338,98.837384,73.519402,16.249293,37.775184,76.532525,33.314515,3.0,99.242952,49.767798,100.0,15.0,1,1.0,0.0,0.0,1.0,14.0,2
3,54.488318,10.204481,96.218855,61.555956,17.960368,74.866362,8.110139,8.0,8.500107,42.783438,90.0,33.0,1,1.0,0.0,0.0,1.0,11.0,3
4,42.365480,20.887676,24.875314,12.381998,2.467873,90.371974,40.724117,5.0,9.445791,82.255945,68.0,23.0,1,0.0,1.0,1.0,0.0,6.0,4


In [29]:
synthetic_dataset = []
for i in range(100):
    tmp = generate_patient(description_file)
    tmp.loc[:, 'stay_id'] = (i+1)*1000
    synthetic_dataset.append(tmp.copy())
synthetic_dataset = pd.concat(synthetic_dataset)

/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alw

/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alw

/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alw

/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  synthetic_dataset.loc[:, 'hospital_expire_flag'] = 1
/var/folders/fq/459_c0qd28l6fqslyzgx45p40000gn/T/ipykernel_1264/289768591.py:30: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of alw

In [30]:
synthetic_dataset.head()

,rate_epinephrine,rate_norepinephrine,rate_dopamine,rate_dobutamine,meanbp_min,pao2fio2ratio_novent,pao2fio2ratio_vent,gcs_min,bilirubin_max,creatinine_max,platelet_min,admission_age,hospital_expire_flag,icu_expire_flag,admission_type_Medical,admission_type_ScheduledSurgical,admission_type_UnscheduledSurgical,charlson_comorbidity_index,hr,stay_id
0,54.881350,35.997806,77.869540,37.303486,71.376687,47.938455,47.582620,7.0,63.784030,26.334652,44.0,27.0,1,1.0,1.0,0.0,0.0,11.0,0,1000
1,71.518937,72.999056,77.740756,52.497044,63.918690,91.486309,70.877039,3.0,9.789600,93.011077,85.0,90.0,1,0.0,1.0,0.0,0.0,6.0,1,1000
2,60.276338,17.162968,25.942256,75.059502,39.916115,4.934895,4.397543,3.0,26.091105,37.344889,72.0,34.0,1,1.0,0.0,0.0,0.0,14.0,2,1000
3,54.488318,52.103661,37.381314,33.350747,43.176013,29.288857,87.952148,10.0,17.792968,77.163213,38.0,70.0,1,0.0,0.0,1.0,1.0,12.0,3,1000
4,42.365480,5.433799,58.759964,92.415877,61.452770,71.505260,52.008142,13.0,62.005358,33.984304,42.0,41.0,1,1.0,1.0,1.0,1.0,10.0,4,1000


In [31]:
synthetic_dataset['icu_expire_flag'].describe()

count    10678.000000
mean         0.487732
std          0.499873
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max          1.000000
Name: icu_expire_flag, dtype: float64

In [32]:
synthetic_dataset['hospital_expire_flag'].describe()

count    10678.0
mean         1.0
std          0.0
min          1.0
25%          1.0
50%          1.0
75%          1.0
max          1.0
Name: hospital_expire_flag, dtype: float64

In [33]:
synthetic_dataset.to_csv('synthetic_dataset.csv')